In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta  

In [3]:
#pretend to be a browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [4]:
#url data
base_url = 'https://www.procyclingstats.com/race/'

In [19]:
#grand tours
tours = ['giro-d-italia','tour-de-france','vuelta-a-espana']
#considered years
years = list(range(2010,2017,1))
years = [str(x) for x in years]
#stages
stages = list(range(1,22,1))
stages = [str(x) for x in stages]

In [51]:
#Pull grand tour data
tour = tours[0]
year = str(years[0])
stage = str(stages[0])
url = base_url + tour +'/'+ year + '/stage-' + stage
r = requests.get(url,headers = header)
results = pd.read_html(r.text)
results = results[0]

#add fields
results['race'] = tour
results['year'] = year
results['stage'] = stage
results.columns = results.columns.str.lower()

In [40]:
#fix time
time = [x[0] for x in results.time.str.split()]
for n,t in enumerate(time):
    if t == ',,':
        time[n] = '00:00:00'
    elif len(t) == 4:
        time[n] = '00:0' + t
    elif len(t) == 5:
        time[n] = '00:' + t
    elif len(t) == 7:
        time[n] = '0' + t
time = pd.to_datetime(time,format='%H:%M:%S')    
results.time = Series(time).dt.time

In [9]:
cum_time = []
for n,t in enumerate(time):
    if n == 0:
        cum_time.append(time[n])
    else:
        hours = time[n].hour
        minutes = time[n].minute
        seconds = time[n].second
        ct = cum_time[n-1] + timedelta(hours = hours, minutes = minutes, seconds = seconds)
        cum_time.append(ct)
results['cum_time'] = Series(cum_time).dt.time

In [71]:
def fix_time(t):
    '''fix formatting of each time element'''
    #remove duplicates:
    t = t.split()[0]
    #turn into time:
    if t == ',,':
        t = '00:00:00'
    elif len(t) == 4:
        t = '00:0' + t
    elif len(t) == 5:
        t = '00:' + t
    elif len(t) == 7:
        t = '0' + t
    t = pd.to_datetime(t,format='%H:%M:%S')
    return t    

In [82]:
def cum_time(time):
    '''time is our list of times'''
    cum_time = []
    for n,t in enumerate(time):
        if n == 0:
            cum_time.append(time[n])
        else:
            hours = time[n].hour
            minutes = time[n].minute
            seconds = time[n].second
            ct = cum_time[n-1] + timedelta(hours = hours, minutes = minutes, seconds = seconds)
            cum_time.append(ct)
    return(cum_time)

In [ ]:
data = pd.DataFrame()
for year in years:
    for tour in tours:
        for stage in stages:
            url = base_url + tour +'/'+ year + '/stage-' + stage
            r = requests.get(url,headers=header)
            results = pd.read_html(r.text)
            results = results[0]
            results['race'] = tour
            results['year'] = year
            results['stage'] = stage
            if data.empty:
                data = results
            else:
                #data = 
                data.append(results)

In [79]:
#test stage site
url = 'https://www.cyclingstage.com/giro-2016-route/'
r = requests.get(url,headers = header)
results = pd.read_html(r.text)

In [ ]:
#Pull grand tour data
tour = tours[0]
year = str(years[0])
stage = str(stages[0])
url = base_url + tour +'/'+ year + '/stage-' + stage
r = requests.get(url,headers = header)
results = pd.read_html(r.text)
results = results[0]

#add fields
results['race'] = tour
results['year'] = year
results['stage'] = stage
results.columns = results.columns.str.lower()

In [32]:
data

,Rnk,BIB,Rider,Age,Team,UCI,Pnt,Avg,Time,GC,GC-Time,race,year,stage
0,1,171,Wiggins Bradley,30,Sky Procycling,16.0,80.0,48.932,10:18 10:18,1.0,+0:00,giro-d-italia,2010,1
1,2,2,Bookwalter Brent,26,BMC Racing Team,8.0,50.0,48.774,0:02 0:02,NaN,+0:00,giro-d-italia,2010,1
2,3,1,Evans Cadel,33,BMC Racing Team,4.0,35.0,48.774,",, 0:02",NaN,+0:00,giro-d-italia,2010,1
3,4,41,Vinokourov Alexandre,36,Astana,2.0,25.0,48.539,0:05 0:05,NaN,+0:00,giro-d-italia,2010,1
4,5,177,Henderson Gregory,33,Sky Procycling,1.0,18.0,48.539,",, 0:05",NaN,+0:00,giro-d-italia,2010,1
5,6,219,Porte Richie,25,Saxo Bank,NaN,15.0,48.539,",, 0:05",NaN,+0:00,giro-d-italia,2010,1
6,7,111,Millar David,33,Team Garmin - Transitions,NaN,12.0,48.462,0:06 0:06,NaN,+0:00,giro-d-italia,2010,1
7,8,215,Larsson Gustav,29,Saxo Bank,NaN,10.0,48.384,0:07 0:07,NaN,+0:00,giro-d-italia,2010,1
8,9,168,van Emden Jos,25,Rabobank ProTeam,NaN,8.0,48.230,0:09 0:09,NaN,+0:00,giro-d-italia,2010,1
9,10,182,Pinotti Marco,34,Team HTC - Columbia,NaN,6.0,48.230,",, 0:09",NaN,+0:00,giro-d-italia,2010,1


In [85]:
results[0]

,0,1,2,3,4
0,NaN,Date,Start and finish,Length,Type
1,1.0,Fr 6-5,Giro 2016 stage 1 Apeldoorn (NL) – Apeldoorn,9.8 km,ITT
2,2.0,Sa 7-5,Giro 2016 stage 2 Arnhem (NL) – Nijmegen (NL),190.0 km,flat
3,3.0,Su 8-5,Giro 2016 stage 3 Nijmegen (NL) – Arnhem (NL),190.0 km,flat
4,NaN,Mo 9-5,rest day / transfer,NaN,NaN
5,4.0,Tu 10-5,Giro 2016 stage 4 Catanzaro – Praia a Mare,200.0 km,hilly
6,5.0,We 11-5,Giro 2016 stage 5 Praia a Mare – Bienvenito,233.0 km,hilly
7,6.0,Tu 12-5,Giro 2016 stage 6 Ponte – Roccaraso,157.0 km,mountains
8,7.0,Fr 13-5,Giro 2016 stage 7 Sulmano – Foligno,210.0 km,"hilly, flat finale"
9,8.0,Sa 14-5,Giro 2016 stage 8 Foligno – Arezzo,186.0 km,hilly
